# Exploring Hacker News Posts

Our aim in this project is to analyze what is the feedback for two types of HN posts (Ask and show).

We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

Let's start by importing the libraries we need and reading the data set into a list of lists.

In [1]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers. Let's remove that first row next.

In [2]:
headers = hn[0]
hn = hn[1:]

print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

To find the posts that begin with either Ask HN or Show HN, we'll use the string method startswith.

If we wish to control for case, we can use the lower method which returns a lowercase version of the starting string.

Let's use these methods to separate posts beginning with Ask HN and Show HN (and case variations) into two different lists next.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


Next, let's determine if ask posts or show posts receive more comments on average.

In [4]:
total_ask_comments = 0

for post in ask_posts:
    total_ask_comments += int(post[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)

print(avg_ask_comments)

14.038417431192661


In [5]:
total_show_comments = 0

for post in show_posts:
    total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts)

print(avg_show_comments)

10.31669535283993


As we can see, Ask HN has on average more comments than Show HN.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

Next, we'll tackle the first step — calculating the amount of ask posts and comments by hour created. We'll use the datetime module to work with the data in the created_at column.

In [6]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append([row[6], int(row[4])])
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    date = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = dt.datetime.strftime(date, "%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

In the last cell, we created two dictionaries:

- counts_by_hour: contains the number of ask posts created during each hour of the day.
- comments_by_hour: contains the corresponding number of comments ask posts created at each hour received.

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

Let's create a list of lists containing the hours during which posts were created and the average number of comments those posts received.

In [7]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['18', 13.20183486238532],
 ['12', 9.41095890410959],
 ['21', 16.009174311926607],
 ['19', 10.8],
 ['14', 13.233644859813085],
 ['11', 11.051724137931034],
 ['08', 10.25],
 ['15', 38.5948275862069],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['10', 13.440677966101696],
 ['16', 16.796296296296298],
 ['00', 8.127272727272727],
 ['07', 7.852941176470588],
 ['02', 23.810344827586206],
 ['04', 7.170212765957447],
 ['09', 5.5777777777777775],
 ['22', 6.746478873239437],
 ['23', 7.985294117647059],
 ['06', 9.022727272727273],
 ['17', 11.46],
 ['20', 21.525],
 ['13', 14.741176470588234],
 ['01', 11.383333333333333]]

In the last cell, we calculated the average number of comments for posts created during each hour of the day, and stored the results in a list of lists named avg_by_hour.

Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [17]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)
sorted_swap

print("Top 5 Hours for Ask Post Comment")

for avg, hour in sorted_swap[:5]:
    hr_dt = dt.datetime.strptime(hour, "%H")
    hr_str = hr_dt.strftime("%H:%M")
    print("{}: {:.2f} average comments per post".format(hr_str, avg))

[[13.20183486238532, '18'], [9.41095890410959, '12'], [16.009174311926607, '21'], [10.8, '19'], [13.233644859813085, '14'], [11.051724137931034, '11'], [10.25, '08'], [38.5948275862069, '15'], [7.796296296296297, '03'], [10.08695652173913, '05'], [13.440677966101696, '10'], [16.796296296296298, '16'], [8.127272727272727, '00'], [7.852941176470588, '07'], [23.810344827586206, '02'], [7.170212765957447, '04'], [5.5777777777777775, '09'], [6.746478873239437, '22'], [7.985294117647059, '23'], [9.022727272727273, '06'], [11.46, '17'], [21.525, '20'], [14.741176470588234, '13'], [11.383333333333333, '01']]
Top 5 Hours for Ask Post Comment
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


# Conclusions

As we can see, the top hour is hour 15 (3pm EST). We can use this to maximize the amount of comments that are aquired when making a post on Hacker Noon in the Ask HN catergory.